<a href="https://www.kaggle.com/code/anthonynam/mobile-price-prediction-sklearn-gradient-boosting?scriptVersionId=140232550" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Install Library

In [1]:
!pip install scikit-learn

In [2]:
!pip install numpy==1.22.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 48.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
chex 0.1.81 requires numpy>=1.25.0, but you have numpy 1.22.4 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.22.4 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.1 which is incompatible.
ydata-profiling 4.3.1 requires scipy<1.11,>=1.4.1, but you have scipy 1.11.1 which is incompatible.


## import library

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, average_precision_score,f1_score
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold,GridSearchCV

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load Dataset

In [4]:
train_ds = pd.read_csv("/kaggle/input/mobile-price-classification/train.csv")
train_ds.head(5)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
test_ds = pd.read_csv("/kaggle/input/mobile-price-classification/test.csv")
test_ds.head(5)

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,...,16,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,...,12,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,...,4,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,...,20,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,...,18,749,810,1773,15,8,7,1,0,1


# Pre-training Statistical Analysis

In [6]:
# Check statistics of price range
print(train_ds[["price_range"]].describe())

       price_range
count  2000.000000
mean      1.500000
std       1.118314
min       0.000000
25%       0.750000
50%       1.500000
75%       2.250000
max       3.000000


In [7]:
# Check distribution of result
print(train_ds["price_range"].value_counts())

# If the labels are imbalanced, we can use balanced_accuracy_score in sklearn_metrics

1    500
2    500
3    500
0    500
Name: price_range, dtype: int64


## Prepare data

In [8]:
X = train_ds.loc[:,train_ds.columns != "price_range"]
y = train_ds.loc[:,train_ds.columns == "price_range"]

X_train_all, X_test,y_train_all,y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)
X_train, X_val, y_train,y_val = train_test_split(X_train_all,y_train_all, test_size = 0.2, random_state = 42)

print(f"X_train_all shape: {X_train_all.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")
print("")
print(f"y_train_all shape: {y_train_all.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"y_test shape: {y_test.shape}")

X_train_all shape: (1600, 20)
X_train shape: (1280, 20)
X_val shape: (320, 20)
X_test shape: (400, 20)

y_train_all shape: (1600, 1)
y_train shape: (1280, 1)
y_val shape: (320, 1)
y_test shape: (400, 1)


## scikit-learn Gradient Boosting Classifier (Without Cross-validation)

In [9]:
# Model Training and Fitting
sk_gbc = GradientBoostingClassifier()
sk_gbc.fit(X_train,y_train.values.ravel())

GradientBoostingClassifier()

In [10]:
# Valiation
y_val_pred = sk_gbc.predict(X_val)

# Model performance evaluation
print(f"Accuracy Score (Validation Set): {accuracy_score(y_val,y_val_pred)}")
print(f"Weighted F1 Score (Validation Set): {f1_score(y_val,y_val_pred,average = 'weighted')}")

Accuracy Score (Validation Set): 0.90625
Weighted F1 Score (Validation Set): 0.9057706359678637


In [11]:
# Test with unseen data
y_test_pred = sk_gbc.predict(X_test)

# Model performance evaluation
print(f"Accuracy Score (Test Set): {accuracy_score(y_test,y_test_pred)}")
print(f"Weighted F1 Score (Test Set): {f1_score(y_test,y_test_pred,average = 'weighted')}")

Accuracy Score (Test Set): 0.8925
Weighted F1 Score (Test Set): 0.8934481834813478


## scikit-learn Gradient Boosting Classifier (With Cross-validation)

In [12]:
# 5-fold cross validation
cv = RepeatedStratifiedKFold(n_splits = 5,n_repeats = 3, random_state = 42)
sk_gbc = GradientBoostingClassifier(learning_rate = 0.15)
scores = cross_val_score(sk_gbc, X_train_all, y_train_all.values.ravel(), cv=cv,scoring = "accuracy")
print(f"10-fold Accuracy: {scores}")
print(f"Mean Accuracy: {np.mean(scores)}")
print(f"Stdev Accuracy: {np.std(scores)}")

# Fit the model the all training data
sk_gbc.fit(X_train_all,y_train_all.values.ravel())

10-fold Accuracy: [0.909375 0.878125 0.921875 0.903125 0.88125  0.896875 0.8875   0.896875
 0.915625 0.90625  0.9      0.9      0.884375 0.890625 0.91875 ]
Mean Accuracy: 0.8993749999999999
Stdev Accuracy: 0.013020416659999784


GradientBoostingClassifier(learning_rate=0.15)

In [13]:
# Test with unseen data
y_test_pred = sk_gbc.predict(X_test)

# Model performance evaluation
print(f"Accuracy Score (Test Set): {accuracy_score(y_test,y_test_pred)}")
print(f"Weighted F1 Score (Test Set): {f1_score(y_test,y_test_pred,average = 'weighted')}")

Accuracy Score (Test Set): 0.91
Weighted F1 Score (Test Set): 0.9102118183943219


## scikit-learn Gradient Boosting Classifier (With Grid Search + Cross Validation)

In [14]:
# parameters 
parameters = {'n_estimators':[10,20,50,100,150,200], 'learning_rate':[0.01,0.02,0.5,0.1,0.2]}

# 5-fold cross validation
sk_gbc = GradientBoostingClassifier()
sk_gbc = GridSearchCV(estimator = sk_gbc,param_grid = parameters,cv = 10,verbose = 1,n_jobs = -1)

# Fit the model the all training data
sk_gbc.fit(X_train_all,y_train_all.values.ravel())

Fitting 10 folds for each of 30 candidates, totalling 300 fits


GridSearchCV(cv=10, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.5, 0.1, 0.2],
                         'n_estimators': [10, 20, 50, 100, 150, 200]},
             verbose=1)

In [15]:
sk_gbc.cv_results_

{'mean_fit_time': array([0.32483535, 0.64948049, 1.68643787, 3.20758946, 4.99085858,
        6.37784083, 0.32720821, 0.64253271, 1.59173002, 3.32392712,
        4.84687705, 6.356161  , 0.32098927, 0.64882588, 1.76774697,
        3.1521039 , 4.73781877, 6.48964503, 0.32151871, 0.6367559 ,
        1.60717003, 3.18306911, 4.93808889, 6.35751662, 0.32233164,
        0.6466228 , 1.58780959, 3.28553946, 4.84879458, 6.12411244]),
 'std_fit_time': array([0.00147496, 0.00697254, 0.06960372, 0.02595473, 0.25995364,
        0.03095422, 0.01242214, 0.00669819, 0.01288933, 0.18603233,
        0.12391456, 0.0265318 , 0.00247223, 0.02542067, 0.22038552,
        0.01189844, 0.02478258, 0.2641493 , 0.00199791, 0.00379572,
        0.01988995, 0.01914   , 0.21006946, 0.0882751 , 0.00156624,
        0.02161318, 0.00596587, 0.19277191, 0.18130893, 0.38824871]),
 'mean_score_time': array([0.0043236 , 0.00434411, 0.00484781, 0.00578692, 0.00688696,
        0.0079458 , 0.00422935, 0.00438087, 0.00498397, 0.00

In [16]:
print(sk_gbc.best_estimator_)
print(sk_gbc.best_score_)
print(sk_gbc.best_params_)

GradientBoostingClassifier(learning_rate=0.5)
0.905
{'learning_rate': 0.5, 'n_estimators': 100}


In [17]:
# Test with unseen data
y_test_pred = sk_gbc.predict(X_test)
y_test_proba = sk_gbc.predict_log_proba(X_test)

# Model performance evaluation
print(f"Accuracy Score (Test Set): {accuracy_score(y_test,y_test_pred)}")
print(f"Weighted F1 Score (Test Set): {f1_score(y_test,y_test_pred,average = 'weighted')}")

Accuracy Score (Test Set): 0.91
Weighted F1 Score (Test Set): 0.9098912173346307


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:1387: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [18]:
print(y_test_proba)

[[-4.11072443e-05 -1.01015885e+01 -1.62017368e+01 -2.33395791e+01]
 [-1.17310111e+01 -5.93532109e+00 -2.66924752e-03 -1.12296929e+01]
 [-8.49864145e+00 -2.10441740e-04 -1.19370329e+01 -1.58474288e+01]
 ...
 [-1.35920289e+01 -4.17482517e-03 -5.48436635e+00 -1.11975554e+01]
 [-1.59538726e+01 -3.48843812e-04 -7.96274905e+00 -1.45694034e+01]
 [-1.57162709e+01 -1.12231582e+01 -1.23463589e-03 -6.70860658e+00]]
